<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-documentation/tree/master/ImageCollection/04_mapping_over_image_collection.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-documentation/blob/master/ImageCollection/04_mapping_over_image_collection.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-documentation/master?filepath=ImageCollection/04_mapping_over_image_collection.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-documentation/blob/master/ImageCollection/04_mapping_over_image_collection.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping over an ImageCollection
To apply a function to every `Image` in an `ImageCollection` use `imageCollection.map()`. The only argument to `map()` is a function which takes one parameter: an `ee.Image`. For example, the following code adds a timestamp band to every image in the collection:

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

# Import libraries    
import ee
import folium
import geehydro

# Authenticate and initialize Earth Engine API
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [2]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [3]:
# This function adds a band representing the image timestamp.
def addTime(image): 
  return image.addBands(image.metadata('system:time_start'))

# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
  .filter(ee.Filter.eq('WRS_PATH', 44)) \
  .filter(ee.Filter.eq('WRS_ROW', 34))

first = collection.map(addTime).first()

# Map the function over the collection and display the result.
# print(collection.map(addTime).getInfo())
print(first.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA', 'system:time_start']


For one-line fuction, you can also use a [lambda function](https://www.w3schools.com/python/python_lambda.asp), which is a small anonymous function. 

In [4]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
  .filter(ee.Filter.eq('WRS_PATH', 44)) \
  .filter(ee.Filter.eq('WRS_ROW', 34))

first = collection.map(lambda image: image.addBands(image.metadata('system:time_start'))).first()

# Map the function over the collection and display the result.
# print(collection.map(addTime).getInfo())
print(first.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'BQA', 'system:time_start']


Note that in the predefined function, the `metadata()` method is used to create a new `Image` from the value of a property. As discussed in the [Reducing](https://developers.google.com/earth-engine/ic_reducing) and [Compositing](https://developers.google.com/earth-engine/ic_composite_mosaic) sections, having that time band is useful for the linear modeling of change and for making composites.

The mapped function is limited in the operations it can perform. Specifically, it can’t modify variables outside the function; it can’t print anything; it can’t use JavaScript ‘if’ or ‘for’ statements. However, you can use `ee.Algorithms.If()` to perform conditional operations in a mapped function. For example:

In [5]:
# This function uses a conditional statement to return the image if
# the solar elevation > 40 degrees.  Otherwise it returns a zero image.
def conditional(image):
  return ee.Algorithms.If(ee.Number(image.get('SUN_ELEVATION')).gt(40), image, ee.Image(0))

# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
  .filter(ee.Filter.eq('WRS_PATH', 44)) \
  .filter(ee.Filter.eq('WRS_ROW', 34))

# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA') \
  .filter(ee.Filter.eq('WRS_PATH', 44)) \
  .filter(ee.Filter.eq('WRS_ROW', 34))

# Map the function over the collection, convert to a List and print the result.
# print('Expand this to see the result: ', collection.map(conditional).getInfo())

Inspect the list of images in the output ImageCollection and note that the when the condition evaluated by the `If()` algorithm is true, the output contains a constant image. Although this demonstrates a server-side conditional function (learn more about client vs. server in Earth Engine on [this page](https://developers.google.com/earth-engine/client_server)), avoid `If()` in general and use filters instead.

In [6]:
# a function for calculating NDVI for Landsat 8 imagery
def calNDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

point = ee.Geometry.Point(-83.93, 35.85)

# select three Landsat 8 imagery in the Knoxville area with the least cloud coverage
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA') \
    .filterBounds(point) \
    .sort('CLOUD_COVER') \
    .limit(3)
    
# calculate NDVI for each image using the map fucntion    
ndvi_images = collection.map(calNDVI)

vizParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0, 
    'max': 0.4
}

Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

Map.centerObject(point, 8)

for i in range(0, 3):
    landsat = ee.Image(collection.toList(3).get(i))
    ndvi = ee.Image(ndvi_images.toList(3).get(i))
   
    Map.addLayer(landsat, vizParams, 'Landsat image ' + str(i+1))
    Map.addLayer(ndvi, {'palette': ['red', 'green']}, 'NDVI image ' + str(i+1))
    
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

Use the lambda function 

In [7]:
point = ee.Geometry.Point(-83.93, 35.85)

# select three Landsat 8 imagery in the Knoxville area with the least cloud coverage
collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA') \
    .filterBounds(point) \
    .sort('CLOUD_COVER') \
    .limit(3)
    
# calculate NDVI for each image using the map fucntion    
ndvi_images = collection.map(lambda image: image.normalizedDifference(['B5', 'B4']))

vizParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0, 
    'max': 0.4
}

Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

Map.centerObject(point, 8)

for i in range(0, 3):
    landsat = ee.Image(collection.toList(3).get(i))
    ndvi = ee.Image(ndvi_images.toList(3).get(i))
   
    Map.addLayer(landsat, vizParams, 'Landsat image ' + str(i+1))
    Map.addLayer(ndvi, {'palette': ['red', 'green']}, 'NDVI image ' + str(i+1))
    
Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map